ex6:  
cv score : 0.01582363008925492  
cv score : 0.014583733652430035  

ex7:  
prodしたら良さそうだった  
CVを書き直しましょう  
かき直した  
cv score : 0.01696220617131911  
cv score : 0.015633094022344574  

0.01564

In [ ]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline



In [ ]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
seeds = [0, 1, 2, 3, 4, 5, 6]
SCALE = "quantile"

    

In [ ]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
#pub_test_df = pd.read_csv("../input/moapublictest/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
y_nonv = y[mask].reset_index(drop=True)#
y_all_nonv = y_all[mask].reset_index(drop=True)

scored = y_nonv.copy()#
y_nonv.drop("drug_id", axis=1, inplace=True)#
y.drop("drug_id", axis=1, inplace=True)#

TR_NONV_SIZE = train_df.shape[0]
TE_NONV_SHAPE = test_df.shape[0]

In [ ]:
# product columns
# あるdrug idで活性、不活性が顕著な特徴の平均

prod_cols = [['g-145', 'g-201', 'g-208'], ['g-370', 'g-508', 'g-37'], ['g-38', 'g-392', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-181', 'g-100', 'g-392'], ['g-67', 'g-760', 'g-50'], ['g-731', 'g-100', 'g-707'], ['g-478', 'g-468', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-106', 'g-744', 'g-91'], ['g-131', 'g-208', 'g-392'], ['g-144', 'g-123', 'g-86'], ['g-228', 'g-72', 'g-67'], ['g-31', 'g-328', 'g-460'], ['g-392', 'g-731', 'g-100'], ['g-732', 'g-744', 'g-707'], ['g-705', 'g-375', 'g-704'], ['g-508', 'g-50', 'g-411'], ['g-234', 'g-58', 'g-520'], ['g-503', 'g-761', 'g-50'], ['g-113', 'g-75', 'g-178'], ['g-50', 'g-508', 'g-113'], ['g-113', 'g-375', 'g-75'], ['g-576', 'g-452', 'g-392'], ['g-50', 'g-37', 'g-36'], ['g-707', 'g-133', 'g-392'], ['g-484', 'g-392', 'g-544'], ['g-508', 'g-67', 'g-370'], ['g-123', 'g-731', 'g-100'], ['g-298', 'g-477', 'g-644'], ['g-72', 'g-370', 'g-50'], ['g-67', 'g-178', 'g-113'], ['g-744', 'g-608', 'g-100'], ['g-91', 'g-100', 'g-707'], ['g-37', 'g-228', 'g-202'], ['g-37', 'g-300', 'g-370'], ['g-234', 'g-508', 'g-595'], ['g-596', 'g-744', 'g-707'], ['g-300', 'g-227', 'g-591'], ['g-135', 'g-392', 'g-512'], ['g-731', 'g-744', 'g-158'], ['g-69', 'g-707', 'g-100'], ['g-276', 'g-653', 'g-291'], ['g-624', 'g-615', 'g-189'], ['g-181', 'g-707', 'g-38'], ['g-72', 'g-75', 'g-508'], ['g-231', 'g-707', 'g-392'], ['g-508', 'g-37', 'g-72'], ['g-725', 'g-712', 'g-640'], ['g-67', 'g-644', 'g-113'], ['g-508', 'g-228', 'g-656'], ['g-185', 'g-37', 'g-672'], ['g-370', 'g-50', 'g-503'], ['g-201', 'g-745', 'g-599'], ['g-332', 'g-50', 'g-571'], ['g-50', 'g-37', 'g-59'], ['g-508', 'g-113', 'g-231'], ['g-707', 'g-158', 'g-100'], ['g-257', 'g-50', 'g-72']]
#prod_cols = [    ['g-712',  'g-208',  'g-38',  'g-100',  'g-123',  'g-328',  'g-744',  'g-248',  'g-460',  'g-731',  'g-417',  'g-349',  'g-131'], ['g-228',  'g-75',  'g-67',  'g-760',  'g-37',  'g-406',  'g-50',  'g-672',  'g-63',  'g-72',  'g-195'], ['g-100', 'g-157', 'g-178'],['g-183', 'g-300', 'g-767'],['g-50', 'g-37', 'g-489', 'g-257', 'g-332'],['g-270', 'g-135', 'g-231', 'g-158', 'g-478', 'g-146', 'g-491', 'g-392'],['g-745', 'g-635', 'g-235'], ['g-300', 'g-414', 'g-62', 'g-34'], ['g-91', 'g-392'], ['g-75', 'g-113'], ['g-599', 'g-261', 'g-38', 'g-146', 'g-392', 'g-512', 'g-744'], ['g-50', 'g-332', 'g-37', 'g-58', 'g-705'], ['g-157', 'g-178'],['g-759', 'g-100', 'g-167', 'g-75', 'g-431', 'g-189', 'g-522', 'g-91'],['g-202', 'g-385', 'g-769'],           ]


for cols in prod_cols:
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)

PRODS = [col for col in train_df.columns if col.startswith("prod-")]

In [ ]:
#out fold preprocessing

#variance threshold
# variance encodeする意味は、分散が少ない→目的変数の分離に貢献しなそうな特徴の削除、無理やりrankgaussする意味はないように思われる
VAR_THRESHOLD = 0.8
drop_cols = []
temp = pd.concat([train_df, pub_test_df])
for col in BIOS+PRODS:
    if temp[col].var() <= VAR_THRESHOLD:
        drop_cols.append(col)

print("drop cols num : {}".format(len(drop_cols)))
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)
pub_test_df.drop(columns=drop_cols, inplace=True)

GENES_ = [col for col in train_df.columns if col.startswith("g-")]
CELLS_ = [col for col in train_df.columns if col.startswith("c-")]
BIOS_ = GENES_ + CELLS_
PRODS = [col for col in train_df.columns if col.startswith("prod-")]
del temp

# onehot encode of categorical feature and drop
drop_cols = ["cp_time", "cp_dose", "time_dose"]
train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot", drop_first=True), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot", drop_first=True), test_df.drop(drop_cols, axis=1) ], axis=1)
pub_test_df = pd.concat([pd.get_dummies(pub_test_df["time_dose"], prefix="onehot", drop_first=True), pub_test_df.drop(drop_cols, axis=1) ], axis=1)

# aggregation feature
# なぜかスケールそのままの方がCVが良くなる
print("agg")

for df in [train_df, pub_test_df, test_df]:
    df["agg-sum-g"] = (df[GENES_].sum(axis=1) - df[GENES_].sum(axis=1).min() + 1).map(np.log)
    df["agg-mean-g"] = (df[GENES_].mean(axis=1) - df[GENES_].mean(axis=1).min() + 1).map(np.log)
    df["agg-std-g"] = df[GENES_].std(axis=1).map(np.log)
    df["agg-kurt-g"] = (df[GENES_].kurt(axis=1) - df[GENES_].kurt(axis=1).min() + 1).map(np.log)
    df["agg-skew-g"] = (df[GENES_].skew(axis=1) - df[GENES_].skew(axis=1).min() + 1).map(np.log)
    df["agg-sum-c"] = (df[CELLS_].sum(axis=1) - df[CELLS_].sum(axis=1).min() +1).map(np.log)
    df["agg-mean-c"] = (df[CELLS_].mean(axis=1) - df[CELLS_].mean(axis=1).min() +1).map(np.log)
    df["agg-std-c"] = df[CELLS_].std(axis=1).map(np.log)
    df["agg-kurt-c"] = (df[CELLS_].kurt(axis=1) - df[CELLS_].kurt(axis=1).min() +1).map(np.log)
    df["agg-skew-c"] = (df[CELLS_].skew(axis=1) - df[CELLS_].skew(axis=1).min() +1).map(np.log)
    df["agg-sum-gc"] = (df[BIOS_].sum(axis=1) - df[BIOS_].sum(axis=1).min() + 1).map(np.log)
    df["agg-mean-gc"] = (df[BIOS_].mean(axis=1) - df[BIOS_].mean(axis=1).min() + 1).map(np.log)
    df["agg-std-gc"] = df[BIOS_].std(axis=1).map(np.log)
    df["agg-kurt-gc"] = (df[BIOS_].kurt(axis=1) - df[BIOS_].kurt(axis=1).min() + 1).map(np.log)
    df["agg-skew-gc"] = (df[BIOS_].skew(axis=1) - df[BIOS_].skew(axis=1).min() + 1).map(np.log)

"""
for df in [train_df, pub_test_df, test_df]:
    df["agg-sum-g"] = df[GENES_].sum(axis=1)
    df["agg-mean-g"] = df[GENES_].mean(axis=1)
    df["agg-std-g"] = df[GENES_].std(axis=1)
    df["agg-kurt-g"] = df[GENES_].kurt(axis=1)
    df["agg-skew-g"] = df[GENES_].skew(axis=1)
    df["agg-sum-c"] = df[CELLS_].sum(axis=1)
    df["agg-mean-c"] = df[CELLS_].mean(axis=1)
    df["agg-std-c"] = df[CELLS_].std(axis=1)
    df["agg-kurt-c"] = df[CELLS_].kurt(axis=1)
    df["agg-skew-c"] = df[CELLS_].skew(axis=1)
    df["agg-sum-gc"] = df[BIOS_].sum(axis=1)
    df["agg-mean-gc"] = df[BIOS_].mean(axis=1)
    df["agg-std-gc"] = df[BIOS_].std(axis=1)
    df["agg-kurt-gc"] = df[BIOS_].kurt(axis=1)
    df["agg-skew-gc"] = df[BIOS_].skew(axis=1)
"""

AGG = [col for col in train_df.columns if col.startswith("agg-")]

In [ ]:
X = train_df.drop("sig_id", axis=1)
y_nonv = y_nonv.drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values
y_all_nonv = y_all_nonv.drop("sig_id", axis=1).values

## Dateset Class

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

## func 

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        # if cycle
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)


        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

## Model

In [ ]:
"""
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size=1500):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.25)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
"""

class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
"""
    
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden1 = 1024
        self.hidden2 =  512
        self.hidden3 = 512
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, self.hidden1))
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden1)
        self.dropout2 = nn.Dropout(0.45)
        self.dense2 = nn.utils.weight_norm(nn.Linear(self.hidden1, self.hidden2))
        
        self.batch_norm3 = nn.BatchNorm1d(self.hidden2)
        self.dropout3 = nn.Dropout(0.35)
        self.dense3 = nn.utils.weight_norm(nn.Linear(self.hidden2, self.hidden3))
        
        self.batch_norm4 = nn.BatchNorm1d(self.hidden3)
        self.dropout4 = nn.Dropout(0.45)
        self.dense4 = nn.utils.weight_norm(nn.Linear(self.hidden3, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x
"""

## run train

In [ ]:
def run_training(model, trainloader, validloader, tag, epochs, optimizer, scheduler, fine_tune_scheduler, loss_fn, loss_tr, early_stopping_steps, verbose, device, fold, seed):
    
    early_step = 0
    best_loss = np.inf
    best_epoch = 0
    weight_path = 'dnn_weights2/{}_{}_{}.pt'.format(tag, seed, fold)
    
    start = time.time()
    t = time.time() - start
    for epoch in range(epochs):
        # fine tune 
        if fine_tune_scheduler is not None:
            fine_tune_scheduler.step(epoch, model)
            
        train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, device)
        valid_loss = valid_fn(model, loss_fn, validloader, device)

        if epoch % verbose==0:
            t = time.time() - start
            print(f"EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}, time: {t}")
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(),weight_path)
            early_step = 0
            best_epoch = epoch
        
        elif early_stopping_steps != 0:
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                t = time.time() - start
                print(f"early stopping in iteration {epoch},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
                return model
            
    print(f"training until max epoch {epochs},  : best itaration is {best_epoch}, valid loss is {best_loss}, time: {t}")
    return model
            
    
def predict(model, testloader, device):
    model.to(device)
    predictions = inference_fn(model, testloader, device)
    
    return predictions

In [ ]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
"""
class FineTuneScheduler:
    def __init__(self, epochs, layer_num):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []
        self.layer_num = layer_num

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = int(name.split('.')[0][-1])

            if layer_index == self.layer_num:  #最後の層以外は凍結する
                continue

            param.requires_grad = False
            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)
        print(self.frozen_layers)
        #self.epochs_per_step = self.epochs // len(self.frozen_layers)+1  # 24 // 4 = 6
        #self.epochs_per_step = self.epochs // (len(self.frozen_layers)+1)  # 24 // 4 = 6
        self.epochs_per_step = 6
        
        # Replace the top layers with another ones, 最後に追加されてく
        model_new.batch_norm4 = nn.BatchNorm1d(model.hidden3)
        model_new.dropout4 = nn.Dropout(0.45)
        model_new.dense4 = nn.utils.weight_norm(nn.Linear(model.hidden3, num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0 or epoch == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = int(name.split('.')[0][-1])

                if layer_index == last_frozen_index:
                    print(epoch, "   ",name)
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen
"""
class FineTuneScheduler:
    def __init__(self, epochs, layer_num):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []
        self.layer_num = layer_num

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = int(name.split('.')[0][-1])

            if layer_index == self.layer_num:  #最後の層以外は凍結する
                continue

            param.requires_grad = False
            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)
        print(self.frozen_layers)
        #self.epochs_per_step = self.epochs // len(self.frozen_layers)+1  # 24 // 4 = 6
        #self.epochs_per_step = self.epochs // (len(self.frozen_layers)+1)  # 24 // 4 = 6
        self.epochs_per_step = 5
        
        # Replace the top layers with another ones, 最後に追加されてく
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0 or epoch == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = int(name.split('.')[0][-1])

                if layer_index == last_frozen_index:
                    print(epoch, "   ",name)
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

## Training by Fold

In [ ]:
BATCH_SIZE = 128
DEVICE = ('cuda:3' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EPOCHS = 25
#EPOCHS = 24
EARLY_STOPPING_STEPS = 10

train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_df.shape[0], y_nonv.shape[1]))
imps = []
imp_cols = []
folds = []
test_cv_preds = []

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    
    ###############################################################################################
    # LOAD LIBRARIES
    targets = SCORED_MOAS.copy()

    # LOCATE DRUGS
    vc = scored["drug_id"].value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values} # drug id がどのフォールドに属すか格納
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.loc[scored["drug_id"].isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp["sig_id"][idxV].values}
        dct2.update(dd)

    # ASSIGN K
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored["fold"].isna(),'fold'] = scored.loc[scored["fold"].isna(),'sig_id'].map(dct2)
    scored["fold"] = scored["fold"].astype('int8')
    ###############################################################################################

    #for fold, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):    
    for fold in range(K):
        train_index = scored[scored["fold"] != fold].index.to_list()
        valid_index = scored[scored["fold"] == fold].index.to_list()
        print("======================== fold {} ========================".format(fold+1))
        folds.append(train_index)
                
        # split data
        train_X = X.iloc[train_index]
        train_y = y_nonv[train_index]
        train_y_all = y_all_nonv[train_index]
        valid_X = X.iloc[valid_index]
        valid_y = y_nonv[valid_index]
        valid_y_all = y_all_nonv[valid_index]
        test_X = (test_df.drop("sig_id", axis=1))
        pub_test_X = (pub_test_df.drop("sig_id", axis=1))

        
        ### scaler ##########
        # validの分布を知らせてはいけない
        print(SCALE)
        scale_cols = BIOS_+PRODS
        scaler = make_scaler(SCALE, seed).fit(train_X.append(pub_test_X)[scale_cols])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[scale_cols] = scaler.transform(df[scale_cols])

    
        ### PCA ##########
        # validの分布を知らせてはいけない
        print("PCA")
        n_decom_g = 80 # 80
        n_decom_c = 10 # 10
        decom_g_cols = [f"pca_g-{i}" for i in range(n_decom_g)]
        decom_c_cols = [f"pca_c-{i}" for i in range(n_decom_c)]
        
        pca_g = PCA(n_components = n_decom_g, random_state = seed).fit(train_X.append(pub_test_X)[GENES_])
        pca_c = PCA(n_components = n_decom_c, random_state = seed).fit(train_X.append(pub_test_X)[CELLS_])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[decom_g_cols] = pca_g.transform(df[GENES_])
            df[decom_c_cols] = pca_c.transform(df[CELLS_])
            
            

            
            
        # prepare data for training
        train_X = train_X.values
        valid_X = valid_X.values
        test_X = test_X.values
        print(train_X.shape[1])
        
        
        # ================================model training===========================
        train_dataset = MoADataset(train_X, train_y_all)
        valid_dataset = MoADataset(valid_X, valid_y_all)
        test_dataset = TestDataset(test_X)
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
        testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
        
        train_dataset2 = MoADataset(train_X, train_y)
        valid_dataset2 = MoADataset(valid_X, valid_y)
        trainloader2 = torch.utils.data.DataLoader(train_dataset2, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        validloader2 = torch.utils.data.DataLoader(valid_dataset2, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

        model = Model(
            num_features=train_X.shape[1],
            num_targets=train_y_all.shape[1],
        )

        model.to(DEVICE)
        
        
        optimizer = torch.optim.Adam( model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY,)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader) )
        fine_tune_scheduler = FineTuneScheduler(EPOCHS, 5)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=1e-3)
        
        # pre train
        model = run_training(
            model=model,
            trainloader=trainloader,
            validloader=validloader,
            tag="ALL",
            epochs=EPOCHS,
            optimizer=optimizer,
            scheduler=scheduler,
            fine_tune_scheduler=None,
            loss_fn=loss_fn,
            loss_tr=loss_tr,
            early_stopping_steps=EARLY_STOPPING_STEPS,
            device=DEVICE,
            verbose=5,
            fold=fold,
            seed=seed,)
        
        model.load_state_dict(torch.load('dnn_weights2/ALL_{}_{}.pt'.format(seed, fold)), DEVICE)
        model = fine_tune_scheduler.copy_without_top(model, train_X.shape[1], train_y_all.shape[1], train_y.shape[1])
        
        # train
        optimizer = torch.optim.Adam( model.parameters(), lr=LEARNING_RATE, weight_decay=3e-6, eps=1e-6)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,pct_start=0.1, div_factor=1e2, max_lr=3e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader) )
        model = run_training(
            model=model,
            trainloader=trainloader2,
            validloader=validloader2,
            tag="SCORED",
            epochs=EPOCHS,
            optimizer=optimizer,
            scheduler=scheduler,
            fine_tune_scheduler=fine_tune_scheduler,
            loss_fn=loss_fn,
            loss_tr=loss_tr,
            early_stopping_steps=EARLY_STOPPING_STEPS,
            device=DEVICE,
            verbose=5,
            fold=fold,
            seed=seed,)
        
        
        model.load_state_dict(torch.load('dnn_weights2/SCORED_{}_{}.pt'.format(seed, fold)), DEVICE)
        
        #valid predict
        val_preds = predict(
            model=model,
            testloader=validloader,
            device=DEVICE,)
        
        #test predict
        test_preds = predict(
            model=model,
            testloader=testloader,
            device=DEVICE)
        # ================================model training===========================

        train_pred[valid_index] +=  val_preds
        preds += test_preds / (K*len(seeds))

    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

In [ ]:
train_preds2 = np.zeros((TR_SIZE,  y.shape[1]))
train_preds2[train_nonvehicle_index] = train_preds


preds2 = np.zeros((TE_SIZE, y.shape[1]))
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
#sub_df.to_csv("submission.csv", index=False)

In [ ]:
train_sub = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
cols = [col for col in train_sub.columns if col != "sig_id"]
train_sub[cols] = train_preds2
train_sub.to_csv("dnn_train_preds.csv", index=False)

In [ ]:
!zip dnn_weights2.zip dnn_weights2/S*